## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather-database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,48.99,88,100,1.99,overcast clouds
1,1,Souillac,MU,-20.5167,57.5167,75.20,100,75,8.05,moderate rain
2,2,Kavieng,PG,-2.5744,150.7967,78.37,77,100,13.29,light rain
3,3,Vaini,TO,-21.2000,-175.2000,73.40,88,20,4.61,few clouds
4,4,Mahebourg,MU,-20.4081,57.7000,75.20,100,75,8.05,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?50
What is the maximum temperature you would like for your trip?91


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Souillac,MU,-20.5167,57.5167,75.20,100,75,8.05,moderate rain
2,2,Kavieng,PG,-2.5744,150.7967,78.37,77,100,13.29,light rain
3,3,Vaini,TO,-21.2000,-175.2000,73.40,88,20,4.61,few clouds
4,4,Mahebourg,MU,-20.4081,57.7000,75.20,100,75,8.05,moderate rain
5,5,Punta Arenas,CL,-53.1500,-70.9167,50.00,53,75,25.32,broken clouds
6,6,Mataura,NZ,-46.1927,168.8643,50.00,95,100,3.00,light rain
7,7,Rikitea,PF,-23.1203,-134.9692,78.57,69,7,10.38,clear sky
8,8,Castro,BR,-24.7911,-50.0119,70.75,86,58,4.74,broken clouds
11,11,Hasaki,JP,35.7333,140.8333,50.00,87,75,4.61,broken clouds
13,13,Hilo,US,19.7297,-155.0900,77.00,61,40,6.91,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                472
City                   472
Country                466
Lat                    472
Lng                    472
Max Temp               472
Humidity               472
Cloudiness             472
Wind Speed             472
Current Description    472
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(inplace=True)
clean_df.count()

City_ID                466
City                   466
Country                466
Lat                    466
Lng                    466
Max Temp               466
Humidity               466
Cloudiness             466
Wind Speed             466
Current Description    466
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Souillac,MU,75.20,moderate rain,-20.5167,57.5167,
2,Kavieng,PG,78.37,light rain,-2.5744,150.7967,
3,Vaini,TO,73.40,few clouds,-21.2000,-175.2000,
4,Mahebourg,MU,75.20,moderate rain,-20.4081,57.7000,
5,Punta Arenas,CL,50.00,broken clouds,-53.1500,-70.9167,
6,Mataura,NZ,50.00,light rain,-46.1927,168.8643,
7,Rikitea,PF,78.57,clear sky,-23.1203,-134.9692,
8,Castro,BR,70.75,broken clouds,-24.7911,-50.0119,
11,Hasaki,JP,50.00,broken clouds,35.7333,140.8333,
13,Hilo,US,77.00,scattered clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index , row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f'{lat},{lng}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url , params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found ... skipping.')
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not 

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df['Hotel Name'].dropna(inplace=True)
clean_hotel_df.count()

City                   466
Country                466
Max Temp               466
Current Description    466
Lat                    466
Lng                    466
Hotel Name             466
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))